In [ ]:
!wget -c "http://www.phontron.com/download/conala-corpus-v1.1.zip"


--2021-03-21 12:07:30--  http://www.phontron.com/download/conala-corpus-v1.1.zip
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52105440 (50M) [application/zip]
Saving to: ‘conala-corpus-v1.1.zip’

conala-corpus-v1.1. 100%[===================>]  49.69M  84.7MB/s    in 0.6s    

2021-03-21 12:07:30 (84.7 MB/s) - ‘conala-corpus-v1.1.zip’ saved [52105440/52105440]



In [ ]:
!unzip -n conala-corpus-v1.1.zip

Archive:  conala-corpus-v1.1.zip
   creating: conala-corpus/
  inflating: conala-corpus/conala-mined.jsonl  
  inflating: conala-corpus/conala-train.json  
  inflating: conala-corpus/conala-test.json  


In [ ]:
!head conala-corpus/conala-mined.jsonl

{"parent_answer_post_id": 34705233, "prob": 0.8690001442846342, "snippet": "sorted(l, key=lambda x: (-int(x[1]), x[0]))", "intent": "Sort a nested list by two elements", "id": "34705205_34705233_0", "question_id": 34705205}
{"parent_answer_post_id": 13905946, "prob": 0.8526701436370034, "snippet": "[int(x) for x in str(num)]", "intent": "converting integer to list in python", "id": "13905936_13905946_0", "question_id": 13905936}
{"parent_answer_post_id": 13838041, "prob": 0.8521431843789492, "snippet": "c.decode('unicode_escape')", "intent": "Converting byte string in unicode string", "id": "13837848_13838041_0", "question_id": 13837848}
{"parent_answer_post_id": 23490179, "prob": 0.850829261966626, "snippet": "parser.add_argument('-t', dest='table', help='', nargs='+')", "intent": "List of arguments with argparse", "id": "23490152_23490179_0", "question_id": 23490152}
{"parent_answer_post_id": 2721807, "prob": 0.8403723482242028, "snippet": "datetime.datetime.strptime(s, '%Y-%m-%dT%H:

In [ ]:
!head  conala-corpus/conala-train.json

[
  {
    "intent": "How to convert a list of multiple integers into a single integer?",
    "rewritten_intent": "Concatenate elements of a list 'x' of multiple integers to a single integer",
    "snippet": "sum(d * 10 ** i for i, d in enumerate(x[::-1]))",
    "question_id": 41067960
  },
  {
    "intent": "How to convert a list of multiple integers into a single integer?",
    "rewritten_intent": "convert a list of integers into a single integer",


In [ ]:
import json
snippets = []
f = open("/content/conala-corpus/conala-mined.jsonl")
for i,line in enumerate(f):
  dict_ = json.loads(line)
  snippets.append(dict_['snippet'])

In [ ]:
len(snippets)

593891

In [ ]:
snippets[:5]

['sorted(l, key=lambda x: (-int(x[1]), x[0]))',
 '[int(x) for x in str(num)]',
 "c.decode('unicode_escape')",
 "parser.add_argument('-t', dest='table', help='', nargs='+')",
 "datetime.datetime.strptime(s, '%Y-%m-%dT%H:%M:%SZ')"]

In [ ]:
snippets_split = [ x.split() for x in snippets]

In [ ]:
snippets_split[:5]

[['sorted(l,', 'key=lambda', 'x:', '(-int(x[1]),', 'x[0]))'],
 ['[int(x)', 'for', 'x', 'in', 'str(num)]'],
 ["c.decode('unicode_escape')"],
 ["parser.add_argument('-t',", "dest='table',", "help='',", "nargs='+')"],
 ['datetime.datetime.strptime(s,', "'%Y-%m-%dT%H:%M:%SZ')"]]

In [ ]:
from io import BytesIO
from tokenize import tokenize


def getTokenizer(python_code):
    '''
    Function that returns tokenized python code
    :return: tokenized code
    '''
    tokens = []
    try:
        a = list(tokenize(BytesIO(python_code.encode('utf-8')).readline))
        #print(a)
        for i__ in a[1:-1]:
            if i__.exact_type == 3:
                string_tokens = [k__ for k__ in i__[1]]
                tokens = tokens + string_tokens
            else:
                tokens.append(i__[1])
    except Exception:
        print("Error in tokenization")

    return tokens

In [ ]:
getTokenizer('for i in range(10):\n    print(i)')

['for',
 'i',
 'in',
 'range',
 '(',
 '10',
 ')',
 ':',
 '\n',
 '    ',
 'print',
 '(',
 'i',
 ')',
 '',
 '']

In [ ]:
snippets_tokens = [ getTokenizer(x) for x in snippets]

In [ ]:
snippets_tokens[3]

['parser',
 '.',
 'add_argument',
 '(',
 "'",
 '-',
 't',
 "'",
 ',',
 'dest',
 '=',
 "'",
 't',
 'a',
 'b',
 'l',
 'e',
 "'",
 ',',
 'help',
 '=',
 "'",
 "'",
 ',',
 'nargs',
 '=',
 "'",
 '+',
 "'",
 ')',
 '']

In [ ]:
from gensim.models import Word2Vec
import time

start_time = time.time()

model = Word2Vec(sentences=snippets_tokens, size=100, min_count=2)

print(f'Time taken : {(time.time() - start_time) / 60:.2f} mins')

Time taken : 1.74 mins


In [ ]:
print(model)

Word2Vec(vocab=75541, size=100, alpha=0.025)


In [ ]:
words = list(model.wv.vocab)
print(words[:50])

['sorted', '(', 'l', ',', 'key', '=', 'lambda', 'x', ':', '-', 'int', '[', '1', ']', ')', '0', '', 'for', 'in', 'str', 'num', 'c', '.', 'decode', "'", 'u', 'n', 'i', 'o', 'd', 'e', '_', 's', 'a', 'p', 'parser', 'add_argument', 't', 'dest', 'b', 'help', 'nargs', '+', 'datetime', 'strptime', '%', 'Y', 'm', 'T', 'H']


In [ ]:
print(model['for'])

[-2.2212107  -3.2716973   0.08596397  1.0600432   1.569161    2.6471703
  3.9394183   1.6362194  -1.4283733   0.8181455  -0.40948525  0.79383725
 -0.20749182 -0.6079804   0.67397064 -0.9775447   0.20821586 -0.26805055
 -1.3640004   6.870676    1.8757685   4.8799257  -0.77709275  2.5054526
  1.1443208   2.3336983  -2.3346663   0.35804844 -4.5044103  -1.7924329
 -0.78196406  1.212055    2.8979034   3.057321    2.6749594   4.093399
  1.1332434  -0.66972274 -4.2216635   3.6693451  -3.2459188   1.9979509
 -2.7410555  -1.763092    3.60696     2.7205794  -1.0839537  -2.7054164
 -1.8759485   4.9015856   1.2094648  -0.81035155  0.62556684 -3.4928925
 -1.5139366  -2.3137531   3.1062942  -3.185933   -0.7376263   1.6052458
  2.7609344   0.6255045  -1.845184    3.0349295   0.27142432 -2.362853
 -2.560822   -2.1509025  -1.2535785  -0.17445536 -1.9748359   0.13634776
  2.8430192  -4.3504815   2.530394    0.6075757   3.461263    0.917853
 -1.5209956  -0.5183875  -0.02319751 -0.9546636  -6.0778255  -1.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
model.wv.vector_size


100

In [ ]:
model.wv.most_similar('if')


[('elif', 0.7410098314285278),
 ('or', 0.7151807546615601),
 ('and', 0.6773449182510376),
 ('assert', 0.5952292680740356),
 ('while', 0.5156347751617432),
 ('REPORTLAB22', 0.454201340675354),
 ('yourdatetime', 0.44834327697753906),
 ('arr_none', 0.4468314051628113),
 ('min_diff', 0.44495561718940735),
 ('any', 0.4390013813972473)]

In [ ]:
word_vectors = model.wv

In [ ]:
from gensim.models import KeyedVectors
word_vectors.save('vectors.kv')
reloaded_word_vectors = KeyedVectors.load('vectors.kv')

In [ ]:
reloaded_word_vectors.vocab

{'sorted': <gensim.models.keyedvectors.Vocab at 0x7fa4354be510>,
 '(': <gensim.models.keyedvectors.Vocab at 0x7fa4354be3d0>,
 'l': <gensim.models.keyedvectors.Vocab at 0x7fa4354be590>,
 ',': <gensim.models.keyedvectors.Vocab at 0x7fa4354be550>,
 'key': <gensim.models.keyedvectors.Vocab at 0x7fa4354be650>,
 '=': <gensim.models.keyedvectors.Vocab at 0x7fa4354be690>,
 'lambda': <gensim.models.keyedvectors.Vocab at 0x7fa4354be710>,
 'x': <gensim.models.keyedvectors.Vocab at 0x7fa4354be750>,
 ':': <gensim.models.keyedvectors.Vocab at 0x7fa4354be790>,
 '-': <gensim.models.keyedvectors.Vocab at 0x7fa4354be7d0>,
 'int': <gensim.models.keyedvectors.Vocab at 0x7fa4354be850>,
 '[': <gensim.models.keyedvectors.Vocab at 0x7fa4354be890>,
 '1': <gensim.models.keyedvectors.Vocab at 0x7fa4354be8d0>,
 ']': <gensim.models.keyedvectors.Vocab at 0x7fa4354be910>,
 ')': <gensim.models.keyedvectors.Vocab at 0x7fa4354be950>,
 '0': <gensim.models.keyedvectors.Vocab at 0x7fa4354be990>,
 '': <gensim.models.keyedv

In [ ]:
model.wv.save_word2vec_format('custom_embeddings_300d.txt', binary=False)
model.wv.save()


In [ ]:
model.wv.save_word2vec_format("codeembeddings.bin", binary=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cp custom_embeddings_300d.txt '/content/drive/MyDrive/END/custom_embeddings_300d.txt'

In [ ]:
cp codeembeddings.bin '/content/drive/MyDrive/END/codeembddings.bin'

In [ ]:
import torch
import torch.nn as nn
import torchtext.data as data
import torchtext.vocab as vocab

In [ ]:
import gensim
new_model = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format('/content/custom_embeddings_300d.txt', binary=False)


ValueError: ignored

In [ ]:
fp = open('custom_embeddings_100d.txt', 'r')
x = fp.readlines()

In [ ]:
print(type(x[1]))

<class 'str'>


In [ ]:
x = x[1:]
with open("outfile", "w") as outfile:
    outfile.write("\n".join(str(item) for item in x))

In [ ]:
vectors = vocab.Vectors('outfile')

  0%|          | 0/151083 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
import pandas as pd
X=model[model.wv.vocab]
df=pd.DataFrame(X)
print(df.shape)
df.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(97161, 100)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
97156,0.006755,-0.001741,-0.005286,-0.001270,0.004273,-0.002706,0.009718,-0.005571,0.003631,-0.004235,0.001351,-0.006555,-0.006630,-0.000191,-0.003186,-0.004376,0.000505,-0.001734,0.002627,-0.001021,0.002058,-0.008083,0.000706,0.001907,0.000091,-0.007721,-0.002734,-0.002990,0.002025,0.008075,0.002172,-0.000458,-0.000918,-0.006887,0.000752,0.004644,0.003836,-0.000736,0.001136,-0.008218,...,-0.004293,0.000193,-0.003566,-0.000680,-0.003715,-0.001178,-0.005981,-0.000791,-0.000115,0.004710,0.002197,-0.005148,-0.003680,0.002303,-0.001709,-0.003134,0.008953,0.003432,-0.002353,-0.003599,0.001147,0.002000,0.000909,0.000588,-0.002279,-0.003964,-0.001890,-0.010272,0.000505,0.001547,0.007236,-0.001858,0.003496,-0.000225,0.000166,-0.000785,-0.001607,-0.004333,0.004772,0.004688
97157,0.068957,-0.019370,-0.059231,-0.027206,0.096159,-0.032279,0.089235,0.023693,0.007016,-0.007432,0.021857,-0.035461,-0.011582,0.029440,-0.049529,-0.012746,-0.039042,-0.071235,0.006155,0.040691,-0.019519,-0.039552,-0.001450,0.032188,0.015158,-0.035686,0.026897,-0.034270,-0.010272,0.036000,-0.033513,0.033259,-0.005024,-0.000668,0.027939,0.010390,0.036963,0.040442,0.016186,-0.085641,...,-0.046060,-0.045139,0.009456,-0.004276,0.037403,-0.058625,0.011021,0.026728,-0.016623,0.059258,0.010145,0.001765,0.044789,-0.014717,0.019026,-0.012906,0.050241,0.025463,0.064603,0.006527,0.036100,-0.001235,-0.041555,0.002649,-0.014095,-0.071538,-0.020775,-0.039031,-0.050249,-0.028341,0.040798,-0.008362,0.012145,-0.030084,0.030801,-0.029866,0.002767,-0.079876,0.022345,0.025702
97158,-0.000327,0.002558,-0.004211,-0.000550,0.003121,-0.000478,0.003988,-0.003596,-0.002963,-0.002953,0.000649,0.001378,-0.001450,0.001011,0.002587,0.004064,0.000046,-0.004216,0.002499,0.004374,0.003372,0.001579,0.000098,-0.004780,0.004105,-0.000196,-0.002042,-0.001342,0.004917,-0.002016,-0.004510,-0.002724,-0.000186,0.001576,-0.002714,0.004261,0.002993,-0.002992,-0.002691,-0.002537,...,-0.001115,-0.003945,0.001444,-0.000539,0.000352,0.004881,-0.002884,0.001610,-0.002686,0.003469,-0.001884,-0.002975,0.002367,0.000808,0.004593,-0.000158,0.003389,-0.001946,-0.002834,-0.004799,0.003960,0.000507,-0.002791,0.001882,-0.000249,-0.003140,0.001699,0.002003,0.002062,0.004568,0.003937,0.004169,0.004499,0.002708,0.002345,0.002306,-0.000956,0.000948,-0.001880,-0.001079
97159,0.014532,-0.006153,-0.015459,-0.014090,0.007784,-0.002125,0.021946,-0.002370,0.000577,-0.001396,0.003325,-0.011064,-0.016089,0.008397,-0.000058,-0.002873,-0.006488,-0.020203,0.007871,0.007972,0.007349,-0.001338,0.005498,0.007065,0.008647,-0.013168,-0.002506,0.000874,0.007690,0.009251,-0.002222,0.006261,-0.010255,-0.000163,-0.004151,0.007561,0.014078,0.005816,0.005941,-0.012241,...,-0.011227,-0.001500,0.003523,0.007139,0.004200,-0.018606,0.001636,-0.008833,0.007379,0.008479,0.004662,-0.006083,0.006653,0.003053,0.002801,-0.011309,0.006825,0.008893,0.006277,-0.002745,-0.000250,0.000120,-0.011342,-0.001190,-0.003678,-0.006270,-0.010705,-0.009333,-0.012543,-0.001308,0.003135,0.003540,0.004630,-0.005811,0.010268,-0.005347,-0.006853,-0.007811,0.007252,-0.000293
97160,0.022203,-0.002044,-0.024546,-0.010501,0.005999,0.000596,0.013401,-0.002978,0.007885,0.003933,-0.002082,-0.001433,-0.008162,0.007191,0.001135,-0.005212,-0.010330,-0.016885,-0.002631,0.003747,-0.005577,-0.009979,0.010532,0.009197,0.013577,-0.009849,0.011760,-0.008625,0.004450,0.008904,0.001889,0.000935,-0.000230,0.003656,0.005819,-0.000867,0.008756,-0.001249,0.005098,-0.020209,...,-0.010152,-0.014632,0.007132,0.008080,0.006605,-0.012194,0.003811,-0.000799,-0.011433,0.011257,0.003704,0.003377,0.019686,0.001858,0.004980,-0.004205,0.014665,0.012888,-0.000328,-0.008650,-0.002155,-0.000525,0.003907,-0.001034,-0.009867,-0.018600,-0.002158,-0.009653,-0.003425,-0.003288,-0.00

In [ ]:
import numpy as np
#Computing the correlation matrix
X_corr=df.corr()

#Computing eigen values and eigen vectors
values,vectors=np.linalg.eig(X_corr)

#Sorting the eigen vectors coresponding to eigen values in descending order
args = (-values).argsort()
values = vectors[args]
vectors = vectors[:, args]

#Taking first 2 components which explain maximum variance for projecting
new_vectors=vectors[:,:2]

#Projecting it onto new dimesion with 2 axis
neww_X=np.dot(X,new_vectors)

In [ ]:
!python --version

Python 3.7.10
